In [ ]:
import boto3

In [ ]:
# need to run this from a terminal:
#conda install -c conda-forge poppler

In [ ]:

try:
    from PyPDF2 import PdfFileWriter, PdfFileReader
except:
    !pip install PyPDF2
    from PyPDF2 import PdfFileWriter, PdfFileReader
    
try:
    from pdf2image import convert_from_path
except:
    !pip install pdf2image
    from pdf2image import convert_from_path
    
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

try:
    from trp import Document
except:
    !pip install textract-trp
    from trp import Document

try:
    import tabula
except:
    !pip install tabula-py

!pip install pillow
from PyPDF2 import PdfFileWriter, PdfFileReader

In [ ]:

import pandas as pd
import numpy as np
import json
import boto3
from trp import Document
import difflib

%matplotlib inline

In [ ]:
pdffile=r"f941.pdf"
#r"UC-CR4 Q2 19 AL SUI-sample3.pdf"
#r"AL UC-CR4 Q4 2019-sample1.pdf",
#r"f941.pdf",
#r"Mississippi 89 140 SIT FLT provided.pdf",
#r"NMDOLES903ATotalsReport.pdf",
#r"Oregon OQ Tax Report Q1 2019-sample2.pdf",
#r"TN LB-0456 Q1 18.pdf",
#r"UC-CR4 Q2 19 AL SUI-sample3.pdf",
#r"Handwritten Sample I9.pdf"

## Convert PDF into Images

In [ ]:
pdffile=r"OregonOQ.pdf"
filename=pdffile.split('.')[0]
inputpdf = PdfFileReader(open(pdffile, "rb"))

In [ ]:
!rm -r /home/ec2-user/SageMaker/pdf_pages/
!mkdir -p /home/ec2-user/SageMaker/pdf_pages/

In [ ]:
# convert pdf document into single page files, then upload to s3 

s3 = boto3.client('s3')
BUCKET = "adptextractpoc"
FOLDER = filename

for i in range(inputpdf.numPages):
    output = PdfFileWriter()
    output.addPage(inputpdf.getPage(i))
    with open("/home/ec2-user/SageMaker/pdf_pages/pdf-page%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)

    
for i in range(inputpdf.numPages):    
    images = convert_from_path('/home/ec2-user/SageMaker/pdf_pages/pdf-page{}.pdf'.format(i),  500)
    for image in images:
        image.save('/home/ec2-user/SageMaker/pdf_pages/{}-pdf-page{}.jpg'.format(filename,i), 'JPEG')
        s3.upload_file('/home/ec2-user/SageMaker/pdf_pages/{}-pdf-page{}.jpg'.format(filename,i), 
                       Bucket = BUCKET, 
                       Key = '{}/{}-pdf-page{}.jpg'.format(filename,filename,i))



In [ ]:
# get rid of local pdf pages
!rm -r /home/ec2-user/SageMaker/pdf_pages/

### no need to run

In [ ]:
ttract_client = boto3.client('textract')

In [ ]:
BUCKET = "wsj-tech-internal"

In [ ]:
line = []
conf = []
line_class = []

for i in range(15):
    print(i)
    KEY    = 'textract/pdf-page{}.jpg'.format(i)
    response = ttract_client.detect_document_text(Document={'S3Object':{'Bucket':BUCKET,'Name':KEY}})
    for block in response['Blocks']:
        if block["BlockType"] == "LINE":
            line.append(block.get("Text", "null"))
            conf.append(block.get("Confidence", "null"))
            line_class.append("bad")

In [ ]:
KEY    = 'textract/pdf-page15.jpg'
response = ttract_client.detect_document_text(Document={'S3Object':{'Bucket':BUCKET,'Name':KEY}})

In [ ]:
line_1 = 0
line_2 = 0
for block in response['Blocks']:
    if (line_1 == 1) & (line_2 == 1):
        if block["BlockType"] == "LINE": 
            line.append(block.get("Text", "null"))
            conf.append(block.get("Confidence", "null"))
            line_class.append("suspect")
    if (line_1 == 0) & (line_2 == 0):
        if block["BlockType"] == "LINE":
            line.append(block.get("Text", "null"))
            conf.append(block.get("Confidence", "null"))
            line_class.append("bad")
            if block.get("Text", "null") == "bytch":
                line_1 = 1
    if (line_1 == 1) & (line_2 == 0):
        if block["BlockType"] == "LINE": 
            if block.get("Text", "null") == "comma-separated list.":
                line_2 = 1

In [ ]:
import pandas as pd

df = pd.DataFrame({'Phrase':line, 'Confidence':conf, 'Type':line_class})
df['Confidence'] = df['Confidence'].astype(float)

In [ ]:
pdffile=r"f941.pdf"
#r"UC-CR4 Q2 19 AL SUI-sample3.pdf"
#r"AL UC-CR4 Q4 2019-sample1.pdf",
#r"f941.pdf",
#r"Mississippi 89 140 SIT FLT provided.pdf",
#r"NMDOLES903ATotalsReport.pdf",
#r"Oregon OQ Tax Report Q1 2019-sample2.pdf",
#r"TN LB-0456 Q1 18.pdf",
#r"UC-CR4 Q2 19 AL SUI-sample3.pdf",
#r"Handwritten Sample I9.pdf"

# Textract Production Model

In [ ]:
s3BucketName="adptextractpoc"
documentName="0_AL UC-CR4 Q4 2019-sample1.png"


## 1. raw text extraction

In [ ]:
s3BucketName="adptextractpoc"
#documentName="0_AL UC-CR4 Q4 2019-sample1.png"

pageNo=0
pdffile=r"AL UC-CR4 Q4 2019-sample1.pdf"
filename=pdffile.split('.')[0]
documentName='{}/{}-pdf-page{}.jpg'.format(filename,filename,pageNo)


In [ ]:
import boto3

# Document
#s3BucketName = "ki-textract-demo-docs"
#documentName = "simple-document-image.jpg"

# Amazon Textract client
textract = boto3.client('textract')

# Call Amazon Textract
response = textract.detect_document_text(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    })

#print(response)

# Print detected text
for item in response["Blocks"]:
    if item["BlockType"] == "WORD":
        print ('\033[94m' +  item["Text"] + '\033[0m')

#### write response to json file:

In [ ]:
## write response to json file:
with open('{}-page{}.json'.format(filename,pageNo), 'w') as file:
     file.write(json.dumps(response))

#### load existing json file:

In [ ]:
## load existing json file:
with open('{}-page{}.json'.format(filename,pageNo)) as json_file: 
    response = json.load(json_file)

In [ ]:
## function find Single Word location (X,Y)
WORD2find="35242"
for item in response["Blocks"]:
    if item["BlockType"] == "WORD":
        if item["Text"] == WORD2find:
            print ('\033[94m' +  item["Text"] + '\033[0m')
            
            print(item)
            print(item['Geometry']['Polygon'])
            
            Word_TopLeftX=item['Geometry']['Polygon'][0]['X']
            Word_TopLeftY=item['Geometry']['Polygon'][0]['Y']
            Word_BottomRightX=item['Geometry']['Polygon'][2]['X']
            Word_BottomRightY=item['Geometry']['Polygon'][2]['Y']
            print("TopLeftX:",Word_TopLeftX)
            print("TopLeftY:",Word_TopLeftY)
            print("BottomRightX:",Word_BottomRightX)
            print("BottomRightY:",Word_BottomRightY)

In [ ]:
## function found Line contain certain word(s):
WORDS2find="AL 35242"
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        if WORDS2find in item["Text"]:
            print ('\033[94m' +  item["Text"] + '\033[0m')
            
            print(item['Geometry']['Polygon'])
            thisLoc=item['Geometry']['Polygon']
            
            

In [ ]:
ABC_loc=thisLoc


In [ ]:
AL_loc=thisLoc

In [ ]:
ABC_loc

In [ ]:
AL_loc

In [ ]:
ABC_loc[0]

In [ ]:
AL_loc[3]

In [ ]:
TopLX=ABC_loc[0]['X']
TopLY=ABC_loc[0]['Y']
BotRX=AL_loc[2]['X']
BotRY=AL_loc[2]['Y']

In [ ]:
## function: Give TopLeftX,TopLeftY, BottomRightX,BottomRightY, 
##           find Lines of words(or Words) in between boundingbox:
#TopX=ABC_loc[0]['X']   #TopX of Word:ABC (can use FINDWord or FINDLine to get)
#TopY=ABC_loc[0]['Y']
#BotX=AL_loc[2]['X']    #BottomX of Word:AL  **3rd element is Bottom Right***
#BotY=AL_loc[2]['Y']

findList=[]
for item in response["Blocks"]:
    if item["BlockType"] == "LINE": #can use WORD
        thisTopLX=item['Geometry']['Polygon'][0]['X']
        thisTopLY=item['Geometry']['Polygon'][0]['Y']
        thisBotRX=item['Geometry']['Polygon'][2]['X']
        thisBotRY=item['Geometry']['Polygon'][2]['Y']
        if thisTopLX>=TopLX and thisTopLY>=TopLY and thisBotRX<=BotRX and thisBotRY<=BotRY:
            findList.append(item['Text'])

In [ ]:
findList

## 2. form extraction

### use trp library to parse

In [ ]:
s3BucketName="adptextractpoc"
#documentName="0_AL UC-CR4 Q4 2019-sample1.png"
#documentName="0_AL UC-CR4 Q4 2019-sample1.png"
#documentName="941_2020.png" 

pageNo=0
pdffile=r"NMDOLES903ATotalsReport.pdf"
filename=pdffile.split('.')[0]
documentName='{}/{}-pdf-page{}.jpg'.format(filename,filename,pageNo)
    

In [ ]:
import boto3
from trp import Document

# Document
#s3BucketName = "ki-textract-demo-docs"
#documentName = "employmentapp.png"

# Amazon Textract client
textract = boto3.client('textract')

# Call Amazon Textract
response = textract.analyze_document(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    },
    FeatureTypes=["FORMS"])

#print(response)

doc = Document(response)

form_kv={}
for page in doc.pages:
    # Print fields
    print("Fields:")
    for field in page.form.fields:
        print("Key: {}, Value: {}".format(field.key, field.value))
        try:
            form_kv[field.key.text]=field.value.text
        except:
            form_kv[field.key.text]=None
        
    # Get field by key
#    print("\nGet Field by Key:")
#    key = "EMPLOYER"
#    field = page.form.getFieldByKey(key)
#    if(field):
#        print("Key: {}, Value: {}".format(field.key, field.value))

    # Search fields by key
#    print("\nSearch Fields:")
#    key = "Wages"
#    fields = page.form.searchFieldsByKey(key)
#    for field in fields:
#        print("Key: {}, Value: {}".format(field.key, field.value))

In [ ]:
form_kv

##### save extracted K-V to JSON

In [ ]:
with open('{}-page{}-formkv.json'.format(filename,pageNo), 'w') as file:
     file.write(json.dumps(form_kv))

##### edit the JSON in VScode editor, correct each Key and Value format

In [ ]:
## load pre-defined correct form: Key-Value(format) json file
with open('NMDOLES903ATotalsReport-page0-formkv-standard.json') as json_file: 
    form_kv_standard = json.load(json_file)

In [ ]:
form_kv_standard

In [ ]:
form_kv

In [ ]:
#### function: correct JSON output Key using predefined Key
## input: form_kv - JSON output extracted Key-Value pair dict
##        form_kv_standard - this specific form pre-defined Key-value(format) dict
## function compare each JSON's Key with predefined Key, and use the close-matched pre-defined key

def form_kv_correction(form_kv,form_kv_standard):
    form_kv_corrected={}
    for key in form_kv.keys():
        this_key_correct_matchlist=difflib.get_close_matches(key,form_kv_standard.keys(), n = 1,cutoff = 0.6)
        if len(this_key_correct_matchlist)>0:
            this_key_correct=this_key_correct_matchlist[0]
            form_kv_corrected[this_key_correct]=form_kv[key]
        else:
            form_kv_corrected[key]="NO_MARTCH_KEY_FOUND"
    for key in form_kv_standard.keys():
        if key not in form_kv_corrected.keys():
            form_kv_corrected[key]="NOT_EXTRACTED_KEY"
    return form_kv_corrected

In [ ]:
form_kv_corrected=form_kv_correction(form_kv,form_kv_standard)
form_kv_corrected

In [ ]:
form_kv_standard.keys()


In [ ]:
## sort dictionary by key:
sorted(form_kv_corrected.items(), key = lambda kv:(kv[0]))

In [ ]:
#check float number 
if isinstance(2.3, float): print("YES")   # int for integer 
else: print("NO")
    #check string 
if isinstance('9', str): print("YES")   # int for integer 
else: print("NO")

In [ ]:


#Function: input : ,documentName
#          output: extracted form: dataframe of all Key-Value 
def df_form_keyvalue(s3BucketName,documentName):
    textract = boto3.client('textract')
    response = textract.analyze_document(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    },
    FeatureTypes=["FORMS"])
    
    doc = Document(response)
    
    df_form=pd.DataFrame(columns=['Key','Value'])
    for page in doc.pages:
         for field in page.form.fields:
            df_form = df_form.append({'Key': field.key, 'Value': field.value}, ignore_index=True)
            
    return df_form

In [ ]:
s3BucketName="adptextractpoc"
#documentName="0_AL UC-CR4 Q4 2019-sample1.png"
documentName="941_2020.png"
#documentName="941_2020.png" df_form=df_form_keyvalue(doc)
df_form=df_form_keyvalue(s3BucketName,documentName)
df_form

In [ ]:
    df_form=pd.DataFrame(columns=['Key','Value'])
    for page in doc.pages:
         for field in page.form.fields:
            df_form = df_form.append({'Key': field.key, 'Value': field.value}, ignore_index=True)



In [ ]:
df_form

In [ ]:
a0=df_form.iloc[0]['Value'].text
a1=df_form.iloc[1]['Value'].text
a2=df_form.iloc[2]['Value'].text
a3=df_form.iloc[3]['Value'].text

In [ ]:
a2l=a2.split(" ")
a2l.insert(2,'.')
''.join(a2l)

In [ ]:
#post processing add . inbetween number & remove space in between number 
# e.g.: $ 108. 79 -> $108.79
#       $ 108 79 -> $108.79
#       $ 00     -> $00
def postporocessing_addDecimalPoint(STR):
    nElementSTR=len(STR.split(" "))
    if STR[0]=='$' and nElementSTR>1:
        if nElementSTR==3 and STR.split(" ")[1][-1]=='.': 
            STR=STR.replace(" ","")   #   $ 108. 79 -> $108.79
        elif nElementSTR==3 and STR.split(" ")[1][-1]!='.':
            STRlist=STR.split(" ")
            STRlist.insert(2,'.')
            STR=''.join(STRlist)   # $ 108 79 -> $108.79
        else:
            STR=STR.replace(" ","")  # $ 00 -> $00
    return STR
        

In [ ]:
print(postporocessing_addDecimalPoint(a3))

In [ ]:
## function find closed match string to a list of string:
# import difflib
# print difflib.get_close_matches(
#    "abcd", ["abc", "acd", "abdc", "dcba"])
#can work with strings (other than single words). In this case, you need to lower the cutoff (the default is 0.6), 
#and raise n, the maximum number of matches:

import difflib
print(difflib.get_close_matches(
 "1ST ",                ["3 Net Taxable Wages (Item 1 minus Item 2)", 
                         "10 Penalty Due", 
                         "dcba",
                         "Credit Amount",
                         "1ST MO",
                         "Employee Tax",
                          "Net Tax Due",
                         "2ND MO."
                       ], n = 4,cutoff = 0.6
                        ))

### start from JSON output of Textract

In [ ]:
import boto3
import sys
import re
import json


def get_kv_map(file_name):

    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    client = boto3.client('textract')
    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['FORMS'])

    # Get the text blocks
    blocks=response['Blocks']
    

    # get key and value maps
    key_map = {}
    value_map = {}
    block_map = {}
    for block in blocks:
        block_id = block['Id']
        block_map[block_id] = block
        if block['BlockType'] == "KEY_VALUE_SET":
            if 'KEY' in block['EntityTypes']:
                key_map[block_id] = block
            else:
                value_map[block_id] = block

    return key_map, value_map, block_map


def get_kv_relationship(key_map, value_map, block_map):
    kvs = {}
    for block_id, key_block in key_map.items():
        value_block = find_value_block(key_block, value_map)
        key = get_text(key_block, block_map)
        val = get_text(value_block, block_map)
        kvs[key] = val
    return kvs


def find_value_block(key_block, value_map):
    for relationship in key_block['Relationships']:
        if relationship['Type'] == 'VALUE':
            for value_id in relationship['Ids']:
                value_block = value_map[value_id]
    return value_block


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X '    

                                
    return text


def print_kvs(kvs):
    for key, value in kvs.items():
        print(key, ":", value)


def search_value(kvs, search_key):
    for key, value in kvs.items():
        if re.search(search_key, key, re.IGNORECASE):
            return value

def main(file_name):

    key_map, value_map, block_map = get_kv_map(file_name)

    # Get Key Value relationship
    kvs = get_kv_relationship(key_map, value_map, block_map)
    print("\n\n== FOUND KEY : VALUE pairs ===\n")
    print_kvs(kvs)

    # Start searching a key value
    while input('\n Do you want to search a value for a key? (enter "n" for exit) ') != 'n':
        search_key = input('\n Enter a search key:')
        print('The value is:', search_value(kvs, search_key))



In [ ]:
file_name='941_2020.png'

In [ ]:
key_map, value_map, block_map = get_kv_map(file_name)

In [ ]:
kvs = get_kv_relationship(key_map, value_map, block_map)

In [ ]:
print("\n\n== FOUND KEY : VALUE pairs ===\n")
print_kvs(kvs)

In [ ]:
while input('\n Do you want to search a value for a key? (enter "n" for exit) ') != 'n':
        search_key = input('\n Enter a search key:')
        print('The value is:', search_value(kvs, search_key))

In [ ]:
kvs

In [ ]:
def form_kv_from_JSON(response):
    blocks=response['Blocks']
    key_map = {}
    value_map = {}
    block_map = {}
    for block in blocks:
        block_id = block['Id']
        block_map[block_id] = block
        if block['BlockType'] == "KEY_VALUE_SET":
            if 'KEY' in block['EntityTypes']:
                key_map[block_id] = block
            else:
                value_map[block_id] = block
    kvs = {}
    for block_id, key_block in key_map.items():
        value_block = find_value_block(key_block, value_map)
        key = get_text(key_block, block_map)
        val = get_text(value_block, block_map)
        kvs[key] = val
    return kvs

In [ ]:
form_kv_from_JSON(response)

## 3. Table extraction

In [ ]:
#s3BucketName="adptextractpoc"
#documentName="0_AL UC-CR4 Q4 2019-sample1.png"

s3BucketName="adptextractpoc"
#documentName="0_AL UC-CR4 Q4 2019-sample1.png"
#documentName="0_AL UC-CR4 Q4 2019-sample1.png"
#documentName="941_2020.png" 

pageNo=0
pdffile=r"TN LB-0456 Q1 18.pdf"
filename=pdffile.split('.')[0]
documentName='{}/{}-pdf-page{}.jpg'.format(filename,filename,pageNo)
    
    


### use trp library to parse

In [ ]:
import boto3
from trp import Document

# Document
#s3BucketName = "ki-textract-demo-docs"
#documentName = "expense.png"

# Amazon Textract client
textract = boto3.client('textract')

# Call Amazon Textract
response = textract.analyze_document(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    },
    FeatureTypes=["TABLES"])

#print(response)

doc = Document(response)

def isFloat(input):
  try:
    float(input)
  except ValueError:
    return False
  return True

warning = ""
for page in doc.pages:
     # Print tables
    for table in page.tables:
        for r, row in enumerate(table.rows):
            itemName  = ""
            for c, cell in enumerate(row.cells):
                print("Table[{}][{}] = {}".format(r, c, cell.text))
                if(c == 0):
                    itemName = cell.text
                elif(c == 4 and isFloat(cell.text)):
                    value = float(cell.text)
                    if(value > 1000):
                        warning += "{} is greater than $1000.".format(itemName)
if(warning):
    print("\nReview needed:\n====================\n" + warning)

In [ ]:
#Function: input : s3BucketName,documentName
#          output: list dataframe of all tables  
def df_tables(s3BucketName,documentName):
    textract = boto3.client('textract')

    response = textract.analyze_document(
        Document={
            'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
            }
        },
        FeatureTypes=["TABLES"])


    doc = Document(response)

    df_list= []
    for page in doc.pages:
        
        for table in page.tables:
            this_table=[]
            for r, row in enumerate(table.rows):
                itemName  = ""
                thisrow=[]
                for c, cell in enumerate(row.cells):
                    thisrow.append(cell.text)
                this_table.append(thisrow)
                df = pd.DataFrame(this_table)
            df_list.append(df)
    return df_list

In [ ]:
df=df_tables(s3BucketName,documentName)
print("Form Name :",pdffile)
print("Page No. :",pageNo+1)
print("Number of Tabels:",len(df))

### Correct OregonOQ.pdf page1 table (4 tables)


##### Table 2: unclear format
##### Table 3: can use Form for Key/Value


##### Correct table 4: 

In [ ]:
df_table=pd.DataFrame()
df_table=df[1]
df_table

In [ ]:
#move 1st row as header
df_table.columns = df_table.iloc[0]
df_table=df_table[1:]

# convert dataframe to a list of dict
df_table_dict=df_table.to_dict(orient='records')

# write the list of dict to JSON 
with open('{}-page{}-table4.json'.format(filename,pageNo), 'w') as file:
     json.dump(df_table_dict,file)

# read JSON (table, list of dict), convert to dataFrame 
with open('{}-page{}-table4.json'.format(filename,pageNo)) as json_file: 
    table_json = json.load(json_file)
pd.DataFrame(table_json)

##### Correct Table 1:

In [ ]:
df_table=pd.DataFrame()
df_table=df[2]
df_table

In [ ]:
#move 1st row as header
df_table.columns = df_table.iloc[0]
df_table=df_table[1:]

# convert dataframe to a list of dict
df_table_dict=df_table.to_dict(orient='records')

# write the list of dict to JSON 
with open('{}-page{}-table1.json'.format(filename,pageNo), 'w') as file:
     json.dump(df_table_dict,file)

# read JSON (table, list of dict), convert to dataFrame 
with open('{}-page{}-table1.json'.format(filename,pageNo)) as json_file: 
    table_json = json.load(json_file)
pd.DataFrame(table_json)

### Correct OregonOQ.pdf page2 table (1 table)



In [ ]:
df_table=pd.DataFrame()
df_table=df[0]
df_table

In [ ]:
#move 1st row as header
df_table.columns = df_table.iloc[0]
df_table=df_table[1:]

#drop column 0 - index 
df_table=df_table.drop(df_table.columns[0], axis=1)
df_table

####  need to decide last row (21) 

### Correct TN LB-0456 Q1 18.pdf page1 table (2 tables)



#### correct 1st table

In [ ]:
df_table=df[0]
df_table

In [ ]:
#move 1st row as header
df_table.columns = df_table.iloc[0]
df_table=df_table[1:]

In [ ]:
df_table

#### !!!! date is not correct, need to check either here or later in final output!!!! 

#### correct 2nd table

In [ ]:
df_table=df[1]
df_table

##### !!! need decide the table format (header, columns, index)

#### Correct UC-CR4 Q2 19 AL SUI-sample3.pdf page3 table (1 table)
##### rename column name, write to JSON, and load back as Dataframe

In [ ]:
## Correct UC-CR4 Q2 19 AL SUI-sample3.pdf page 3 table
df_table=df[0]

#move 1st row as header
df_table.columns = df_table.iloc[0]
df_table=df_table[1:]

# rename column 
df_table.columns=["NAME OF WORKER", "WORKERS SOCIAL SECIRITY NUMBER","TOTAL WAGES PAID BEFORE DEDUCTIONS"]

## split Name column into two 
df_table=df[0]

#move 1st row as header
df_table.columns = df_table.iloc[0]
df_table=df_table[1:]

# rename column 
df_table.columns=["NAME OF WORKER", "WORKERS SOCIAL SECIRITY NUMBER","TOTAL WAGES PAID BEFORE DEDUCTIONS"]

#no need to run: add a row
#df_table=df_table.append([pd.Series(["ALEX S", "111-11-1111","3,212.32"], index=df_table.columns )],ignore_index=True)


## split Name column into two : Last Name and First Initial
df_table_name = df_table["NAME OF WORKER"].str.split(expand=True)
df_table_name.columns=["NAME OF WORKER.LAST NAME","NAME OF WORKER.FIRST INITIAL"]
df_table_new=pd.concat([df_table_name,df_table[["WORKERS SOCIAL SECIRITY NUMBER","TOTAL WAGES PAID BEFORE DEDUCTIONS"]]],axis=1)

# convert dataframe to a list of dict
df_table_dict=df_table_new.to_dict(orient='records')

# write the list of dict to JSON 
with open('{}-page{}-table.json'.format(filename,pageNo), 'w') as file:
     json.dump(df_table_dict,file)

# read JSON (table, list of dict), convert to dataFrame 
with open('{}-page{}-table.json'.format(filename,pageNo)) as json_file: 
    table_json = json.load(json_file)
pd.DataFrame(table_json)

In [ ]:
## if need nested dict, create nested dict : Name of Worker

for i in range(len(df_table_dict)):
    df_table_dict[i]['NAME of WORKER']={}
    df_table_dict[i]['NAME of WORKER']['LAST NAME']=df_table_dict[i]['NAME OF WORKER.LAST NAME']
    df_table_dict[i]['NAME of WORKER']['FIRST INITIAL']=df_table_dict[i]['NAME OF WORKER.FIRST INITIAL']
    df_table_dict[i].pop('NAME OF WORKER.LAST NAME')
    df_table_dict[i].pop('NAME OF WORKER.FIRST INITIAL')

In [ ]:
df_table_dict

In [ ]:
with open('{}-page{}-table.json'.format(filename,pageNo), 'w') as file:
     json.dump(df_table_dict,file)

In [ ]:
with open('UC-CR4 Q2 19 AL SUI-sample3-page2-table.json') as json_file: 
    table_json = json.load(json_file)

In [ ]:
pd.DataFrame(table_json)

### start from JSON output of Textract

In [ ]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint


def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                        
                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '    
    return text


def get_table_csv_results(file_name):

    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    # get the results
    client = boto3.client('textract')

    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['TABLES'])

    # Get the text blocks
    blocks=response['Blocks']
#    pprint(blocks)

    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index +1)
        csv += '\n\n'

    return csv

def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)
    
    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():
        
        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'
        
    csv += '\n\n\n'
    return csv

def main(file_name):
    table_csv = get_table_csv_results(file_name)

    output_file = 'output.csv'

    # replace content
    with open(output_file, "wt") as fout:
        fout.write(table_csv)

    # show the results
    print('CSV OUTPUT FILE: ', output_file)

In [ ]:
file_name

In [ ]:
table_csv = get_table_csv_results(file_name)

In [ ]:
print(table_csv)

## Textract Response Parser

###  You can use Textract response parser library to easily parser JSON returned by Amazon Textract. Library parses JSON and provides programming language specific constructs to work with different parts of the document. textractor is an example of PoC batch processing tool that takes advantage of Textract response parser library and generate output in multiple formats.

In [ ]:
s3BucketName="adptextractpoc"
#documentName="0_AL UC-CR4 Q4 2019-sample1.png"
documentName="941_2020.png"
#documentName="0_f941_filll.png" 

textract = boto3.client('textract')

# Call Amazon Textract
response = textract.analyze_document(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    },
    FeatureTypes=["TABLES"]
)


In [ ]:
response

In [ ]:
# Call Amazon Textract and get JSON response
#  client = boto3.client('textract')
#  response = client.analyze_document(Document={...}, FeatureTypes=[...])

# Parse JSON response from Textract
doc = Document(response)

# Iterate over elements in the document
for page in doc.pages:
    # Print lines and words
#    for line in page.lines:
#        print("Line: {}--{}".format(line.text, line.confidence))
#        for word in line.words:
#            print("Word: {}--{}".format(word.text, word.confidence))

    # Print tables
    for table in page.tables:
        for r, row in enumerate(table.rows):
            for c, cell in enumerate(row.cells):
                print("Table[{}][{}] = {}-{}".format(r, c, cell.text, cell.confidence))

    # Print fields
    for field in page.form.fields:
        print("Field: Key: {}, Value: {}".format(field.key.text, field.value.text))

    # Get field by key
    key = "Phone Number:"
    field = page.form.getFieldByKey(key)
    if(field):
        print("Field: Key: {}, Value: {}".format(field.key, field.value))

    # Search fields by key
    key = "address"
    fields = page.form.searchFieldsByKey(key)
    for field in fields:
        print("Field: Key: {}, Value: {}".format(field.key, field.value))

## Latest Development Model

### Table extraction model

In [ ]:
import uuid
import hashlib
import requests
import json
import warnings
 
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
 
 
retry_strategy = Retry(total=3,
                       backoff_factor=0.1,
                       status_forcelist=[500, 502, 503, 504])
 
warnings.filterwarnings('ignore')
class ALICERequestExecutor():
 
    def __init__(self):
        """
        :param logger: logger object required by ALICEClient.
        :param endpoint: string as ALICE endpoint to hit.
        :param region: string as region for S3 resource.
        """
        self._endpoint = 'https://tabledetectortd2-alpha-1500297725.us-east-1.elb.amazonaws.com/theta/invocations'
 
    def _create_payload_data(self, input_bytes):
        request_id = str(uuid.uuid4())
        return {
            'RequestId': request_id,
            'InferenceRequests': [
                {
                    'ImageHash': hashlib.sha256(input_bytes).hexdigest(),
                    'RequestId': request_id,
                }
            ]
        }
 
    def execute(self, input_bytes: bytes) -> bytes:
        """
        Take in image bytes, and Call ALICE client with a new request context.
        :param input_bytes: bytes as image bytes passed to ALICE endpoint.
        :return: bytes as converted response returned from ALICE client.
        """
        session = requests.session()
        session.mount(self.endpoint, HTTPAdapter(max_retries=retry_strategy))
        files = {'uploadfile': input_bytes}
        payload = self._create_payload_data(input_bytes)
        data = {"inference_batch_request": json.dumps(payload)}
        alice_response = session.post(self.endpoint, files=files, data=data, verify=False)
        return alice_response.content
 
    @property
    def endpoint(self):
        return self._endpoint
 
 
if __name__ == '__main__':
    alice_executor = ALICERequestExecutor()
    image_path = '941_2020.png'
    with open(image_path, 'rb') as file:
        bytes = file.read()
    output_bytes  = alice_executor.execute(bytes)
    alice_output = json.loads(output_bytes, encoding='utf-8')
    results = alice_output['InferenceResponses'][0]['Results']
    print(results)

In [ ]:
results[50]

In [ ]:
blocks=results['Blocks']

In [ ]:
doc = Document(results)
for page in doc.pages:
     # Print tables
    for table in page.tables:
        for r, row in enumerate(table.rows):
            itemName  = ""
            for c, cell in enumerate(row.cells):
                print("Table[{}][{}] = {}".format(r, c, cell.text))
                if(c == 0):
                    itemName = cell.text
                elif(c == 4 and isFloat(cell.text)):
                    value = float(cell.text)
                    if(value > 1000):
                        warning += "{} is greater than $1000.".format(itemName)

## Example code: Analyze text

In [ ]:
# https://docs.aws.amazon.com/textract/latest/dg/analyzing-document-text.html
#Analyzes text in a document stored in an S3 bucket. Display polygon box around text and angled text 
import boto3
import io
from io import BytesIO
import sys

import math
from PIL import Image, ImageDraw, ImageFont

def ShowBoundingBox(draw,box,width,height,boxColor):
             
    left = width * box['Left']
    top = height * box['Top'] 
    draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],outline=boxColor)   

def ShowSelectedElement(draw,box,width,height,boxColor):
             
    left = width * box['Left']
    top = height * box['Top'] 
    draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],fill=boxColor)  

# Displays information about a block returned by text detection and text analysis
def DisplayBlockInformation(block):
    print('Id: {}'.format(block['Id']))
    if 'Text' in block:
        print('    Detected: ' + block['Text'])
    print('    Type: ' + block['BlockType'])
   
    if 'Confidence' in block:
        print('    Confidence: ' + "{:.2f}".format(block['Confidence']) + "%")

    if block['BlockType'] == 'CELL':
        print("    Cell information")
        print("        Column:" + str(block['ColumnIndex']))
        print("        Row:" + str(block['RowIndex']))
        print("        Column Span:" + str(block['ColumnSpan']))
        print("        RowSpan:" + str(block['ColumnSpan']))    
    
    if 'Relationships' in block:
        print('    Relationships: {}'.format(block['Relationships']))
    print('    Geometry: ')
    print('        Bounding Box: {}'.format(block['Geometry']['BoundingBox']))
    print('        Polygon: {}'.format(block['Geometry']['Polygon']))
    
    if block['BlockType'] == "KEY_VALUE_SET":
        print ('    Entity Type: ' + block['EntityTypes'][0])
    
    if block['BlockType'] == 'SELECTION_ELEMENT':
        print('    Selection element detected: ', end='')

        if block['SelectionStatus'] =='SELECTED':
            print('Selected')
        else:
            print('Not selected')    
    
    if 'Page' in block:
        print('Page: ' + block['Page'])
    print()

def process_text_analysis(bucket, document):

    #Get the document from S3
    s3_connection = boto3.resource('s3')
                          
    s3_object = s3_connection.Object(bucket,document)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)

    # Analyze the document
    client = boto3.client('textract')
    
    image_binary = stream.getvalue()
    response = client.analyze_document(Document={'Bytes': image_binary},
        FeatureTypes=["TABLES", "FORMS"])
  

    # Alternatively, process using S3 object
    #response = client.analyze_document(
    #    Document={'S3Object': {'Bucket': bucket, 'Name': document}},
    #    FeatureTypes=["TABLES", "FORMS"])

    
    #Get the text blocks
    blocks=response['Blocks']
    width, height =image.size  
    draw = ImageDraw.Draw(image)  
    print ('Detected Document Text')
   
    # Create image showing bounding box/polygon the detected lines/text
    for block in blocks:

        DisplayBlockInformation(block)
             
        draw=ImageDraw.Draw(image)
        if block['BlockType'] == "KEY_VALUE_SET":
            if block['EntityTypes'][0] == "KEY":
                ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height,'red')
            else:
                ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height,'green')  
            
        if block['BlockType'] == 'TABLE':
            ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height, 'blue')

        if block['BlockType'] == 'CELL':
            ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height, 'yellow')
        if block['BlockType'] == 'SELECTION_ELEMENT':
            if block['SelectionStatus'] =='SELECTED':
                ShowSelectedElement(draw, block['Geometry']['BoundingBox'],width,height, 'blue')    
   
            #uncomment to draw polygon for all Blocks
            #points=[]
            #for polygon in block['Geometry']['Polygon']:
            #    points.append((width * polygon['X'], height * polygon['Y']))
            #draw.polygon((points), outline='blue')
            
    # Display the image
    image.show()
    return len(blocks)


def main():
    
    bucket = 'adptextractpoc'
    document = '941_2020.png'


    block_count=process_text_analysis(bucket,document)
    print("Blocks detected: " + str(block_count))
    
if __name__ == "__main__":
    main()

In [ ]:
image

## Example code: Detect text

In [ ]:
#Detects text in a document stored in an S3 bucket. Display polygon box around text and angled text 
import boto3
import io
from io import BytesIO
import sys

import psutil
import time

import math
from PIL import Image, ImageDraw, ImageFont


# Displays information about a block returned by text detection and text analysis
def DisplayBlockInformation(block):
    print('Id: {}'.format(block['Id']))
    if 'Text' in block:
        print('    Detected: ' + block['Text'])
    print('    Type: ' + block['BlockType'])
   
    if 'Confidence' in block:
        print('    Confidence: ' + "{:.2f}".format(block['Confidence']) + "%")

    if block['BlockType'] == 'CELL':
        print("    Cell information")
        print("        Column:" + str(block['ColumnIndex']))
        print("        Row:" + str(block['RowIndex']))
        print("        Column Span:" + str(block['ColumnSpan']))
        print("        RowSpan:" + str(block['ColumnSpan']))    
    
    if 'Relationships' in block:
        print('    Relationships: {}'.format(block['Relationships']))
    print('    Geometry: ')
    print('        Bounding Box: {}'.format(block['Geometry']['BoundingBox']))
    print('        Polygon: {}'.format(block['Geometry']['Polygon']))
    
    if block['BlockType'] == "KEY_VALUE_SET":
        print ('    Entity Type: ' + block['EntityTypes'][0])
    if 'Page' in block:
        print('Page: ' + block['Page'])
    print()

def process_text_detection(bucket, document):

    
    #Get the document from S3
    s3_connection = boto3.resource('s3')
                          
    s3_object = s3_connection.Object(bucket,document)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)

   
    # Detect text in the document
    
    client = boto3.client('textract')
    #process using image bytes                      
    #image_binary = stream.getvalue()
    #response = client.detect_document_text(Document={'Bytes': image_binary})

    #process using S3 object
    response = client.detect_document_text(
        Document={'S3Object': {'Bucket': bucket, 'Name': document}})

    #Get the text blocks
    blocks=response['Blocks']
    width, height =image.size  
    draw = ImageDraw.Draw(image)  
    print ('Detected Document Text')
   
    # Create image showing bounding box/polygon the detected lines/text
    for block in blocks:
            print('Type: ' + block['BlockType'])
            if block['BlockType'] != 'PAGE':
                print('Detected: ' + block['Text'])
                print('Confidence: ' + "{:.2f}".format(block['Confidence']) + "%")

            print('Id: {}'.format(block['Id']))
            if 'Relationships' in block:
                print('Relationships: {}'.format(block['Relationships']))
            print('Bounding Box: {}'.format(block['Geometry']['BoundingBox']))
            print('Polygon: {}'.format(block['Geometry']['Polygon']))
            print()
            draw=ImageDraw.Draw(image)
            # Draw WORD - Green -  start of word, red - end of word
            if block['BlockType'] == "WORD":
                draw.line([(width * block['Geometry']['Polygon'][0]['X'],
                height * block['Geometry']['Polygon'][0]['Y']),
                (width * block['Geometry']['Polygon'][3]['X'],
                height * block['Geometry']['Polygon'][3]['Y'])],fill='green',
                width=2)
            
                draw.line([(width * block['Geometry']['Polygon'][1]['X'],
                height * block['Geometry']['Polygon'][1]['Y']),
                (width * block['Geometry']['Polygon'][2]['X'],
                height * block['Geometry']['Polygon'][2]['Y'])],
                fill='red',
                width=2)    

                 
            # Draw box around entire LINE  
            if block['BlockType'] == "LINE":
                points=[]

                for polygon in block['Geometry']['Polygon']:
                    points.append((width * polygon['X'], height * polygon['Y']))

                draw.polygon((points), outline='black')    
  
                # Uncomment to draw bounding box
                #box=block['Geometry']['BoundingBox']                    
                #left = width * box['Left']
                #top = height * box['Top']           
                #draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],outline='black') 


    # Display the image
    image.show()
    # display image for 10 seconds

    
    return len(blocks)

def main():

    bucket='adptextractpoc'
    document = '941_2020.png'
    block_count=process_text_detection(bucket,document)
    print("Blocks detected: " + str(block_count))
    
if __name__ == "__main__":
    main()


## Test of Tabula

In [ ]:
import tabula

# Read pdf into list of DataFrame
df = tabula.read_pdf(pdffile, pages='all')

In [ ]:
len(df)

In [ ]:
df[1]